# Summary
This document corresponds to Exercise 3 of [this file](https://github.com/PerformanceEstimation/Learning-Performance-Estimation/blob/main/Course.pdf).

If [PEPit](https://pypi.org/project/PEPit/) is not already installed, please execute the following cell.

In [ ]:
!pip install pepit

### Exercise 3.1

This exercise requires no code.

### Exercise 3.2

Codes for computing the worst-case values of $\frac{f(x_N)-f_\star}{\|x_0-x_\star\|^2}$ and $\frac{\min_{0\leqslant i\leqslant N}\{f(x_i)-f_\star\}}{\|x_0-x_\star\|^2}$ for the accelerated gradient method are provided below.

We let the reader adapt the codes for the gradient method and for the heavy-ball method.

In [ ]:
from PEPit import PEP
from PEPit.functions import SmoothConvexFunction

def wc_accelerated_method_lastfunctionvalue(L, n, verbose=1):
    
    # Instantiate PEP
    problem = PEP()

    # Declare a strongly convex smooth function
    func = problem.declare_function(SmoothConvexFunction, L=L)

    # Start by defining its unique optimal point xs = x_* and corresponding function value fs = f_*
    xs = func.stationary_point()
    fs = func(xs)

    # Then define the starting point x0 of the algorithm
    x0 = problem.set_initial_point()

    # Set the initial constraint that is the distance between x0 and x^*
    problem.set_initial_condition((x0 - xs) ** 2 <= 1)

    # Run n steps of the fast gradient method
    x_new = x0
    y = x0
    for i in range(n):
        x_old = x_new
        x_new = y - 1 / L * func.gradient(y)
        y = x_new + i / (i + 3) * (x_new - x_old)

    # Set the performance metric to the function value accuracy
    problem.set_performance_metric(func.value(x_new)-fs)

    # Solve the PEP
    pepit_tau = problem.solve(verbose=verbose)


    # Return the worst-case guarantee of the evaluated method
    return pepit_tau

def wc_accelerated_method_bestfunctionvalue(L, n, verbose=1):
    
    # Instantiate PEP
    problem = PEP()

    # Declare a strongly convex smooth function
    func = problem.declare_function(SmoothConvexFunction, L=L)

    # Start by defining its unique optimal point xs = x_* and corresponding function value fs = f_*
    xs = func.stationary_point()
    fs = func(xs)

    # Then define the starting point x0 of the algorithm
    x0 = problem.set_initial_point()

    # Set the initial constraint that is the distance between x0 and x^*
    problem.set_initial_condition((x0 - xs) ** 2 <= 1)

    # Run n steps of the fast gradient method
    x_new = x0
    y = x0
    for i in range(n):
        x_old = x_new
        # the performance metric corresponds to the min over all specified metrics!
        # here: \min_{i=0,...,N}{ f(x_i)-f_*}
        problem.set_performance_metric(func.value(x_old)-fs) 
        x_new = y - 1 / L * func.gradient(y)
        y = x_new + i / (i + 3) * (x_new - x_old)

    problem.set_performance_metric(func.value(x_new)-fs)

    # Solve the PEP
    pepit_tau = problem.solve(verbose=verbose)


    # Return the worst-case guarantee of the evaluated method
    return pepit_tau

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt

n_max = 30
n_list = list(range(n_max))
L = 1
verbose = 0

pepits_taus_accelerated_last = list()
pepits_taus_accelerated_best = list()

for i in range(n_max):
    t0= time.process_time()
    pepit_tau = wc_accelerated_method_lastfunctionvalue(L, i, verbose)
    pepits_taus_accelerated_last.append(pepit_tau)
    pepit_tau = wc_accelerated_method_bestfunctionvalue(L, i, verbose)
    pepits_taus_accelerated_best.append(pepit_tau)
    t1 = time.process_time() - t0
    print(i+1, '/', n_max,' done (elapsed time:',"%.2f" %t1,'[s])')

In [ ]:
plt.plot(n_list, pepits_taus_accelerated_last, '-', label='[AGM] last f(x_i)-f_\star')
plt.plot(n_list, pepits_taus_accelerated_best, 'o--', label='[AGM] best f(x_i)-f_\star')

plt.legend()
plt.xlabel('N')
plt.ylabel('f(x_i)-f_\star')
plt.loglog()
plt.show()